In [1]:
import pandas as pd
import numpy as np

# 读取数据
df = pd.read_csv('少數股東權益.csv')

# 清理列名
df.columns = df.columns.str.strip()

# 确保 'qut' 列存在
if 'qut' not in df.columns:
    df.reset_index(inplace=True)

# 将宽表转换为长表
df_long = df.melt(id_vars=['qut'], var_name='股票代碼', value_name='数据')

# 生成报告发布日期和财报开始日期
report_dates = {
    '01': '-05-15',
    '02': '-08-14',
    '03': '-11-14',
    '04': '-03-31'  # Q4 的财报在下一年公布
}

def get_report_date(row):
    year, quarter = row['qut'].split('/')
    quarter_str = quarter.zfill(2)
    if quarter_str in report_dates:
        if quarter_str != '04':
            date_str = f"{year}{report_dates[quarter_str]}"
            report_date = pd.to_datetime(date_str)
        else:
            next_year = str(int(year) + 1)
            date_str = f"{next_year}{report_dates[quarter_str]}"
            report_date = pd.to_datetime(date_str)
        return report_date
    else:
        raise ValueError(f"未知的季度编号：{quarter}")

def get_quarter_start_date(row):
    year, quarter = row['qut'].split('/')
    quarter_int = int(quarter)
    month = (quarter_int - 1) * 3 + 1
    date_str = f"{year}-{month:02d}-01"
    return pd.to_datetime(date_str)

df_long['报告发布日期'] = df_long.apply(get_report_date, axis=1)
df_long['季度开始日期'] = df_long.apply(get_quarter_start_date, axis=1)

# 按股票代码和季度排序
df_long = df_long.sort_values(['股票代碼', '季度开始日期']).reset_index(drop=True)

# 设置日期范围
start_date = '2016-10-11'  # 您的开始日期
end_date = '2024-10-11'    # 您的结束日期
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# 创建一个包含所有日期的 DataFrame
df_daily = pd.DataFrame({'日期': date_range})

# 处理每只股票的数据
stock_codes = df_long['股票代碼'].unique()
list_of_dfs = []

for code in stock_codes:
    # 获取当前股票的数据
    df_stock = df_long[df_long['股票代碼'] == code].copy()
    df_stock = df_stock.sort_values('季度开始日期').reset_index(drop=True)

    # 创建一个 DataFrame，用于当前股票的每日数据
    df_stock_daily = df_daily.copy()
    df_stock_daily['股票代碼'] = code

    # 初始化 '数据' 列
    df_stock_daily['数据'] = np.nan

    # 为每个季度生成日期范围，并在日期范围内填充数据
    for i in range(len(df_stock)):
        start_date_quarter = df_stock.loc[i, '报告发布日期']
        if i + 1 < len(df_stock):
            end_date_quarter = df_stock.loc[i + 1, '报告发布日期'] - pd.Timedelta(days=1)
        else:
            end_date_quarter = pd.to_datetime(end_date)
        # 将日期限制在总的日期范围内
        start_date_quarter = max(start_date_quarter, pd.to_datetime(start_date))
        end_date_quarter = min(end_date_quarter, pd.to_datetime(end_date))
        date_range_quarter = pd.date_range(start=start_date_quarter, end=end_date_quarter, freq='D')
        mask = (df_stock_daily['日期'] >= start_date_quarter) & (df_stock_daily['日期'] <= end_date_quarter)
        df_stock_daily.loc[mask, '数据'] = df_stock.loc[i, '数据']

    # 对于开始日期之前的数据，使用上一季度的数据
    first_report_date = df_stock['报告发布日期'].iloc[0]
    if pd.to_datetime(start_date) < first_report_date:
        # 找到第一个有效数据的位置
        initial_data = df_stock['数据'].iloc[0]
        mask = df_stock_daily['日期'] < first_report_date
        df_stock_daily.loc[mask, '数据'] = initial_data

    # 添加到列表中
    list_of_dfs.append(df_stock_daily)

# 合并所有股票的数据
df_all_stocks = pd.concat(list_of_dfs, ignore_index=True)

# 将长表转换为宽表
df_wide = df_all_stocks.pivot(index='日期', columns='股票代碼', values='数据')

# 前向填充缺失值（可选，如果需要）
# df_wide.fillna(method='ffill', inplace=True)

# 重置索引，将日期作为列
df_wide.reset_index(inplace=True)

# 格式化日期为 'YYYY/MM/DD'
df_wide['日期'] = df_wide['日期'].dt.strftime('%Y/%m/%d')
# 查看结果
print(df_wide.head(20))


ValueError: 未知的季度编号：09

In [68]:
df_wide.to_csv('50普通股.csv', index=False)

In [59]:
import pandas as pd

# 假设您的数据存储在 CSV 文件中，您可以根据实际情况修改读取方式

# 读取营收数据
df_revenue = pd.read_csv('50大.csv')
df_revenue_melted = df_revenue.melt(id_vars=['Date'], var_name='ticker', value_name='Adj_Close')

# 读取收盘价数据
df_close = pd.read_csv('50流動負債.csv')
df_close_melted = df_close.melt(id_vars=['Date'], var_name='ticker', value_name='CL')

# 根据日期和股票代碼合并数据
df_merged = pd.merge(df_revenue_melted, df_close_melted, on=['Date', 'ticker'])

# 如果有更多因子，重复上述步骤，然后继续合并
# 例如，读取第三个因子
#df_factor3 = pd.read_csv('50營收.csv')
#df_factor3_melted = df_factor3.melt(id_vars=['Date'], var_name='ticker', value_name='revenue')

# 将新的因子合并到已合并的数据中
#df_merged = pd.merge(df_merged, df_factor3_melted, on=['Date', 'ticker'])


In [70]:
df_factor3 = pd.read_csv('50普通股.csv')
df_factor3_melted = df_factor3.melt(id_vars=['Date'], var_name='ticker', value_name='Common_Stock')

# 将新的因子合并到已合并的数据中
df_merged = pd.merge(df_merged, df_factor3_melted, on=['Date', 'ticker'])


In [71]:
df_merged.to_csv('50合併.csv', index=False)

In [72]:
a = pd.read_csv('50合併.csv')
a['Common_Stock'] = a['Common_Stock']*100

In [73]:
a.to_csv('50合併.csv', index=False)

In [54]:
import pandas as pd
import numpy as np

# 读取数据
df = pd.read_csv('50營收.csv')

# 清理列名
df.columns = df.columns.str.strip()

# 确保 'qut' 列存在
if 'qut' not in df.columns:
    df.reset_index(inplace=True)

# 将宽表转换为长表
df_long = df.melt(id_vars=['qut'], var_name='股票代碼', value_name='数据')

# 生成报告发布日期和财报开始日期
def get_report_date(row):
    try:
        year, month, day = row['qut'].split('/')
        date_str = f"{year}-{month.zfill(2)}-10"
        return pd.to_datetime(date_str)
    except ValueError:
        raise ValueError(f"无效的季度格式：{row['qut']}")

def get_month_start_date(row):
    try:
        year, month, day = row['qut'].split('/')
        date_str = f"{year}-{month.zfill(2)}-01"
        return pd.to_datetime(date_str)
    except ValueError:
        raise ValueError(f"无效的季度格式：{row['qut']}")

df_long['报告发布日期'] = df_long.apply(get_report_date, axis=1)
df_long['月份开始日期'] = df_long.apply(get_month_start_date, axis=1)

# 按股票代码和月份排序
df_long = df_long.sort_values(['股票代碼', '月份开始日期']).reset_index(drop=True)

# 设置日期范围
start_date = '2016-10-11'  # 您的开始日期
end_date = '2024-10-11'    # 您的结束日期
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# 创建一个包含所有日期的 DataFrame
df_daily = pd.DataFrame({'日期': date_range})

# 处理每只股票的数据
stock_codes = df_long['股票代碼'].unique()
list_of_dfs = []

for code in stock_codes:
    # 获取当前股票的数据
    df_stock = df_long[df_long['股票代碼'] == code].copy()
    df_stock = df_stock.sort_values('月份开始日期').reset_index(drop=True)

    # 创建一个 DataFrame，用于当前股票的每日数据
    df_stock_daily = df_daily.copy()
    df_stock_daily['股票代碼'] = code

    # 初始化 '数据' 列
    df_stock_daily['数据'] = np.nan

    # 为每个月的10号生成报告数据，并在报告日期之后保持该数据直到下一个报告日期
    for i in range(len(df_stock)):
        report_date = df_stock.loc[i, '报告发布日期']
        if i + 1 < len(df_stock):
            next_report_date = df_stock.loc[i + 1, '报告发布日期'] - pd.Timedelta(days=1)
        else:
            next_report_date = pd.to_datetime(end_date)
        # 将日期限制在总的日期范围内
        report_date = max(report_date, pd.to_datetime(start_date))
        next_report_date = min(next_report_date, pd.to_datetime(end_date))
        mask = (df_stock_daily['日期'] >= report_date) & (df_stock_daily['日期'] <= next_report_date)
        df_stock_daily.loc[mask, '数据'] = df_stock.loc[i, '数据']

    # 对于开始日期之前的数据，使用第一个报告的数据
    first_report_date = df_stock['报告发布日期'].iloc[0]
    if pd.to_datetime(start_date) < first_report_date:
        # 找到第一个有效数据的位置
        initial_data = df_stock['数据'].iloc[0]
        mask = df_stock_daily['日期'] < first_report_date
        df_stock_daily.loc[mask, '数据'] = initial_data

    # 添加到列表中
    list_of_dfs.append(df_stock_daily)

# 合并所有股票的数据
df_all_stocks = pd.concat(list_of_dfs, ignore_index=True)

# 将长表转换为宽表
df_wide = df_all_stocks.pivot(index='日期', columns='股票代碼', values='数据')

# 前向填充缺失值（可选，如果需要）
df_wide.fillna(method='ffill', inplace=True)

# 重置索引，将日期作为列
df_wide.reset_index(inplace=True)

# 格式化日期为 'YYYY/MM/DD'
df_wide['日期'] = df_wide['日期'].dt.strftime('%Y/%m/%d')
# 查看结果
print(df_wide.head(20))

股票代碼          日期       1101        1216        1301        1303        1326  \
0     2016/10/11  8261376.0  33945525.0  15132776.0  22769489.0  26143482.0   
1     2016/10/12  8261376.0  33945525.0  15132776.0  22769489.0  26143482.0   
2     2016/10/13  8261376.0  33945525.0  15132776.0  22769489.0  26143482.0   
3     2016/10/14  8261376.0  33945525.0  15132776.0  22769489.0  26143482.0   
4     2016/10/15  8261376.0  33945525.0  15132776.0  22769489.0  26143482.0   
5     2016/10/16  8261376.0  33945525.0  15132776.0  22769489.0  26143482.0   
6     2016/10/17  8261376.0  33945525.0  15132776.0  22769489.0  26143482.0   
7     2016/10/18  8261376.0  33945525.0  15132776.0  22769489.0  26143482.0   
8     2016/10/19  8261376.0  33945525.0  15132776.0  22769489.0  26143482.0   
9     2016/10/20  8261376.0  33945525.0  15132776.0  22769489.0  26143482.0   
10    2016/10/21  8261376.0  33945525.0  15132776.0  22769489.0  26143482.0   
11    2016/10/22  8261376.0  33945525.0  15132776.0 

C:\Users\f007213\AppData\Local\Temp\ipykernel_17184\2416540257.py:95: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_wide.fillna(method='ffill', inplace=True)


In [55]:
df_wide.to_csv('營收.csv', index=False)

In [120]:
import pandas as pd
import numpy as np

# 讀取資料
df = pd.read_csv('ROE.csv')

# 清理列名
df.columns = df.columns.str.strip()

# 確保 'qut' 列存在
if 'qut' not in df.columns:
    df.reset_index(inplace=True)

# 將寬表轉換為長表
df_long = df.melt(id_vars=['qut'], var_name='股票代碼', value_name='数据')

# 生成報告發佈日期和財報開始日期
report_dates = {
    '03': '-05-15',
    '06': '-08-14',
    '09': '-11-14',
    '12': '-03-31'  # Q4 的財報在下一年公布
}

def get_report_date(row):
    if isinstance(row['qut'], str):
        year, quarter = row['qut'].split('/')
        quarter_str = quarter.zfill(2)
        if quarter_str in report_dates:
            if quarter_str != '12':
                date_str = f"{year}{report_dates[quarter_str]}"
                report_date = pd.to_datetime(date_str)
            else:
                next_year = str(int(year) + 1)
                date_str = f"{next_year}{report_dates[quarter_str]}"
                report_date = pd.to_datetime(date_str)
            return report_date
        else:
            raise ValueError(f"未知的季度編號：{quarter}")
    else:
        return pd.NaT

def get_quarter_start_date(row):
    if isinstance(row['qut'], str):
        year, quarter = row['qut'].split('/')
        quarter_int = int(quarter)
        month = (quarter_int // 3 - 1) * 3 + 1
        date_str = f"{year}-{month:02d}-01"
        return pd.to_datetime(date_str)
    else:
        return pd.NaT

df_long['报告发布日期'] = df_long.apply(get_report_date, axis=1)
df_long['季度开始日期'] = df_long.apply(get_quarter_start_date, axis=1)

# 去除包含 NaT 的行
df_long.dropna(subset=['报告发布日期', '季度开始日期'], inplace=True)

# 按股票代碼和季度排序
df_long = df_long.sort_values(['股票代碼', '季度开始日期']).reset_index(drop=True)

# 設置日期範圍
start_date = '2016-10-11'  # 您的開始日期
end_date = '2024-10-11'    # 您的結束日期
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# 創建一個包含所有日期的 DataFrame
df_daily = pd.DataFrame({'日期': date_range})

# 處理每只股票的資料
stock_codes = df_long['股票代碼'].unique()
list_of_dfs = []

for code in stock_codes:
    # 获取当前股票的資料
    df_stock = df_long[df_long['股票代碼'] == code].copy()
    df_stock = df_stock.sort_values('季度开始日期').reset_index(drop=True)

    # 創建一個 DataFrame，用於当前股票的每日資料
    df_stock_daily = df_daily.copy()
    df_stock_daily['股票代碼'] = code

    # 初始化 '数据' 列
    df_stock_daily['数据'] = np.nan

    # 為每個季度生成日期範圍，並在日期範圍內填充資料
    for i in range(len(df_stock)):
        start_date_quarter = df_stock.loc[i, '报告发布日期']
        if pd.isna(start_date_quarter):
            continue
        if i + 1 < len(df_stock):
            end_date_quarter = df_stock.loc[i + 1, '报告发布日期'] - pd.Timedelta(days=1)
        else:
            end_date_quarter = pd.to_datetime(end_date)
        # 將日期限制在總的日期範圍內
        start_date_quarter = max(start_date_quarter, pd.to_datetime(start_date))
        end_date_quarter = min(end_date_quarter, pd.to_datetime(end_date))
        if pd.isna(start_date_quarter) or pd.isna(end_date_quarter):
            continue
        date_range_quarter = pd.date_range(start=start_date_quarter, end=end_date_quarter, freq='D')
        mask = (df_stock_daily['日期'] >= start_date_quarter) & (df_stock_daily['日期'] <= end_date_quarter)
        df_stock_daily.loc[mask, '数据'] = df_stock.loc[i, '数据']

    # 對於開始日期之前的資料，使用上一季度的資料
    first_report_date = df_stock['报告发布日期'].iloc[0]
    if pd.to_datetime(start_date) < first_report_date:
        # 找到第一個有效資料的位置
        initial_data = df_stock['数据'].iloc[0]
        mask = df_stock_daily['日期'] < first_report_date
        df_stock_daily.loc[mask, '数据'] = initial_data

    # 添加到列表中
    list_of_dfs.append(df_stock_daily)

# 合併所有股票的資料
df_all_stocks = pd.concat(list_of_dfs, ignore_index=True)

# 將長表轉換為寬表
df_wide = df_all_stocks.pivot(index='日期', columns='股票代碼', values='数据')

# 前向填充缺失值（可選，如果需要）
# df_wide.fillna(method='ffill', inplace=True)

# 重置索引，將日期作為列
df_wide.reset_index(inplace=True)

# 格式化日期為 'YYYY/MM/DD'
df_wide['日期'] = df_wide['日期'].dt.strftime('%Y/%m/%d')
# 查看結果
print(df_wide.head(20))

股票代碼          日期  1101  1216  1301  1303  1326  1402  2002  2207  2301  ...  \
0     2016/10/11   1.8  4.63  3.58   2.5  4.23  1.38  1.99  6.97   2.5  ...   
1     2016/10/12   1.8  4.63  3.58   2.5  4.23  1.38  1.99  6.97   2.5  ...   
2     2016/10/13   1.8  4.63  3.58   2.5  4.23  1.38  1.99  6.97   2.5  ...   
3     2016/10/14   1.8  4.63  3.58   2.5  4.23  1.38  1.99  6.97   2.5  ...   
4     2016/10/15   1.8  4.63  3.58   2.5  4.23  1.38  1.99  6.97   2.5  ...   
5     2016/10/16   1.8  4.63  3.58   2.5  4.23  1.38  1.99  6.97   2.5  ...   
6     2016/10/17   1.8  4.63  3.58   2.5  4.23  1.38  1.99  6.97   2.5  ...   
7     2016/10/18   1.8  4.63  3.58   2.5  4.23  1.38  1.99  6.97   2.5  ...   
8     2016/10/19   1.8  4.63  3.58   2.5  4.23  1.38  1.99  6.97   2.5  ...   
9     2016/10/20   1.8  4.63  3.58   2.5  4.23  1.38  1.99  6.97   2.5  ...   
10    2016/10/21   1.8  4.63  3.58   2.5  4.23  1.38  1.99  6.97   2.5  ...   
11    2016/10/22   1.8  4.63  3.58   2.5  4.23  1.38

In [124]:
df_wide.to_csv('ROE.csv', index=False)

In [112]:
import pandas as pd

# 读取营收数据
df_revenue = pd.read_csv('50合併.csv')

# 读取收盘价数据
df_close = pd.read_csv('50營業收入.csv')

# 将 'ticker' 转换为字符串类型
df_revenue['ticker'] = df_revenue['ticker'].astype(str)

# 转换并融化收盘价数据
df_close_melted = df_close.melt(id_vars=['Date'], var_name='ticker', value_name='SALEQ')
df_close_melted['ticker'] = df_close_melted['ticker'].astype(str)

# 合并数据
df_merged = pd.merge(df_revenue, df_close_melted, on=['Date', 'ticker'])

print(df_merged)

             Date ticker Adj_Close          DLCQ        CSHOQ      PSTKQ  \
0      2016/10/11   1101     18.79  5.885747e+10   3692175900          0   
1      2016/10/11   1216     43.33  1.590000e+11   5682015400          0   
2      2016/10/11   1301     54.96  9.179936e+10   6365740800          0   
3      2016/10/11   1303     42.25  9.052416e+10   7930821600          0   
4      2016/10/11   1326     59.87  9.634099e+10   5861186300          0   
...           ...    ...       ...           ...          ...        ...   
93787  2024/10/11   4904      87.3  5.564809e+10   3605705400          0   
93788  2024/10/11   4938     102.5  2.800000e+11   2663776600          0   
93789  2024/10/11   5871       152  5.760000e+11   1614572400  150000000   
93790  2024/10/11   5880      25.5  7.079145e+09  14709382200          0   
93791  2024/10/11   6505        52  6.619274e+10   9525959700          0   

              DLTTQ         MIBTQ          CHEQ         DVQ  ...  \
0      6.314711e+10

C:\Users\asas9\AppData\Local\Temp\ipykernel_21596\2225348561.py:4: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_revenue = pd.read_csv('50合併.csv')


In [114]:
df_merged.to_csv('50合併.csv', index=False)

In [116]:
a = pd.read_csv('50合併.csv')
a['SALEQ'] = a['SALEQ'].str.replace(',', '')  # 移除逗號
a['SALEQ'] = pd.to_numeric(a['SALEQ'], errors='coerce')  # 轉換為數值型態，處理可能的空值
a['SALEQ'] = a['SALEQ'] * 1000  # 將欄位乘以 1000

C:\Users\asas9\AppData\Local\Temp\ipykernel_21596\889552956.py:1: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  a = pd.read_csv('50合併.csv')


In [23]:
a['CAPXQ']

0        4.589250e+08
1        3.950499e+09
2        1.063772e+09
3        1.299372e+09
4        2.446740e+09
             ...     
93787    1.621816e+09
93788    4.122037e+09
93789    1.949880e+08
93790    2.806760e+08
93791    1.572511e+09
Name: CAPXQ, Length: 93792, dtype: float64

In [118]:
a.to_csv('50合併.csv', index=False)

In [90]:
import pandas as pd
import numpy as np
df = pd.read_csv('50應付帳款.csv')
# 將 '--' 替換為 NaN
df.replace('-', np.nan, inplace=True)

# 將 NaN 替換為 0
df.fillna(0, inplace=True)

print(df)

              日期        1101        1216       1301        1303        1326  \
0     2016/10/11   6,804,530  31,331,326  7,980,918  18,607,770  19,886,158   
1     2016/10/12   6,804,530  31,331,326  7,980,918  18,607,770  19,886,158   
2     2016/10/13   6,804,530  31,331,326  7,980,918  18,607,770  19,886,158   
3     2016/10/14   6,804,530  31,331,326  7,980,918  18,607,770  19,886,158   
4     2016/10/15   6,804,530  31,331,326  7,980,918  18,607,770  19,886,158   
...          ...         ...         ...        ...         ...         ...   
2918  2024/10/07  20,550,266  62,803,006  8,721,911  16,140,401  17,749,924   
2919  2024/10/08  20,550,266  62,803,006  8,721,911  16,140,401  17,749,924   
2920  2024/10/09  20,550,266  62,803,006  8,721,911  16,140,401  17,749,924   
2921  2024/10/10  20,550,266  62,803,006  8,721,911  16,140,401  17,749,924   
2922  2024/10/11  20,550,266  62,803,006  8,721,911  16,140,401  17,749,924   

            1402        2002        2207        230

In [92]:
df.to_csv('50應付帳款.csv', index=False)